### 분석준비

In [1]:
# 기본 라이브러리

# 분석을 위한 라이브러리
import pandas as pd

# 에러메세지 안나오게
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

import plotly.graph_objects as go
import plotly.express as px

from scipy.stats import levene
import scipy.stats

# 그래프 한글 표시
import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False
                                  
# 디비연결
import psycopg2
import json

def rds_python_conn():
    # rds와 python 연동을 위한 정보 입력
    with open('postgres.info.json','r') as json_file:
        db_info = json.load(json_file)

    host = db_info.get('host')
    dbname = 'postgres' 
    user = 'postgres' 
    password = db_info.get('password')
    port = 5432

    connection = psycopg2.connect(host=host,
                                  dbname=dbname ,
                                  user=user ,
                                  password=password ,
                                  port=port)
    return connection

## 제품 사진 수에 따른 A/B TEST

In [2]:
SQL = """
SELECT mql_id,seller_id,business_type,
    CASE
        WHEN lead_type IN ('industry','offline') THEN '오프라인'
        WHEN lead_type IN ('online_medium', 'online_big', 'online_top', 'online_small', 'online_beginner', 'other_publicities') THEN '온라인'
        ELSE 'other'
    END AS "온오프라인"
FROM closed_deals;
"""

closed_deals = pd.read_sql(SQL, rds_python_conn())

In [9]:
SQL = """
SELECT *
FROM products p 
        join order_items o on p.product_id=o.product_id;
"""
products = pd.read_sql(SQL, rds_python_conn())

# closed_deals, products merge
total = products.merge(closed_deals, on='seller_id',how='inner')
total['price_total'] = total['price'] + total['freight_value']
total = total[['business_type','mql_id','price_total','seller_id','order_id','product_category_name','product_name_length','product_description_length','product_photos_qty','온오프라인']]

# 오프라인&manufacturer 판매자만 추출하여 진행.
a=total[(total.온오프라인=='오프라인')&(total.business_type=='manufacturer')]
a.product_description_length.describe()
a

,business_type,mql_id,price_total,seller_id,order_id,product_category_name,product_name_length,product_description_length,product_photos_qty,온오프라인
1267,manufacturer,1104,70.59,20,49812,computers_accessories,45.0,722.0,2.0,오프라인
1268,manufacturer,1104,78.62,20,15975,computers_accessories,50.0,882.0,4.0,오프라인
1269,manufacturer,1104,76.01,20,80107,computers_accessories,45.0,722.0,2.0,오프라인
1270,manufacturer,1104,85.58,20,7025,computers_accessories,54.0,882.0,4.0,오프라인
1271,manufacturer,1104,474.16,20,7163,signaling_and_security,50.0,408.0,3.0,오프라인
...,...,...,...,...,...,...,...,...,...,...
4998,manufacturer,7143,72.18,2714,41700,pet_shop,51.0,433.0,1.0,오프라인
5026,manufacturer,3502,62.87,1632,25493,health_beauty,34.0,966.0,1.0,오프라인
5029,manufacturer,1535,68.35,2541,47743,perfumery,31.0,895.0,1.0,오프라인
5031,manufacturer,3085,57.29,847,7074,baby,42.0,739.0,2.0,오프라인


In [ ]:
a.product_name_length.describe()

## 제품 설명길이

### 가설설정

In [37]:
# (귀무가설) : 오프라인이면서 manufacturer인 사람중에서 제품 설명길이에 따른 매출평균 차이가 없을것.
# (대립가설) : 오프라인이면서 manufacturer인 사람중에서 제품 설명길이에 따른 매출평균 차이가 있을것.

### 등분산 확인

In [44]:
group1_sales = list(a[a.product_description_length>566.500000].groupby('product_category_name')['price_total'].mean())  # 제품설명길이가 566.5초과인것
group2_sales = list(a[a.product_description_length<=566.500000].groupby('product_category_name')['price_total'].mean())  # 제품설명길이가 566.5이하인것

# Levene's test 수행
statistic, p_value = levene(group1_sales, group2_sales)

# 결과 출력
print("Levene's test statistic:", statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("두 그룹의 분산이 통계적으로 유의미하게 다릅니다. 등분산 가정이 만족되지 않을 수 있습니다.")
else:
    print("두 그룹의 분산이 통계적으로 유의미하게 다르지 않습니다. 등분산 가정을 만족할 수 있습니다.")


Levene's test statistic: 0.6395313559655481
p-value: 0.43038271913658377
두 그룹의 분산이 통계적으로 유의미하게 다르지 않습니다. 등분산 가정을 만족할 수 있습니다.


### t-test

In [59]:
# 실제로 이 평균값이 유의미한가를 t-test를 통해 확인


# (귀무가설) : 오프라인이면서 manufacturer인 사람중에서 제품 설명길이에 따른 매출평균 차이가 없을것.
# (대립가설) : 오프라인이면서 manufacturer인 사람중에서 제품 설명길이에 따른 매출평균 차이가 있을것.

group1_sales = list(a[a.product_description_length>566.500000].groupby('product_category_name')['price_total'].mean())  # 사진 수가 1인 그룹의 매출 데이터
group2_sales = list(a[a.product_description_length<=566.500000].groupby('product_category_name')['price_total'].mean())  

# 등분산 가정을 확인하고 독립표본 T-test 수행
statistic, p_value = scipy.stats.ttest_ind(group1_sales, group2_sales, equal_var=True)  # equal_var=False: 등분산 가정이 만족되지 않을 경우 Welch's t-test 사용

# 결과 출력
print("T-statistic:", statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("귀무 가설을 기각하고 대립 가설을 채택합니다.")
else:
    print("귀무 가설을 채택합니다.")

T-statistic: -0.41238638198442445
p-value: 0.6830890047520852
귀무 가설을 채택합니다.


In [60]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=group1_sales, name='제품설명길이<=566', histnorm='density', marker_color='#4A55A2'))
fig.add_trace(go.Histogram(x=group2_sales, name='제품설명길이>566', histnorm='density', marker_color='#A0BFE0'))

fig.update_layout(
    xaxis_title='매출',
    yaxis_title='확률 밀도',
    title='두 그룹의 매출 분포',
    width=800,
    barmode='stack',
)

fig.show()

In [61]:
fig = go.Figure()

# Scatter Trace 추가
fig.add_trace(go.Box(y=group1_sales,name='제품설명길이<=566', marker_color='#4A55A2'))
fig.add_trace(go.Box(y=group2_sales,name='제품설명길이>566', marker_color='#A0BFE0'))
fig.update_layout(
    xaxis_title='제품설명길이',
    yaxis_title='매출',
    title='두 그룹의 매출 분포',
    width=800,
)
fig.show()

## 제품 이름길이

### 가설설정

In [ ]:
# (귀무가설) : 오프라인이면서 manufacturer인 사람중에서 제품 이름길이에 따른 매출평균 차이가 없을것.
# (대립가설) : 오프라인이면서 manufacturer인 사람중에서 제품 이름길이에 따른 매출평균 차이가 있을것.

# 표본 크기가 중요한 이유는 통계적 유의성때문임. 사분위 활용.

### 등분산 확인

In [4]:
group1_sales = list(a[a.product_name_length>52].groupby('product_category_name')['price_total'].mean())  # 제품설명길이가 566.5초과인것
group2_sales = list(a[a.product_name_length<=52].groupby('product_category_name')['price_total'].mean())  # 제품설명길이가 566.5이하인것

# Levene's test 수행
statistic, p_value = levene(group1_sales, group2_sales)

# 결과 출력
print("Levene's test statistic:", statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("두 그룹의 분산이 통계적으로 유의미하게 다릅니다. 등분산 가정이 만족되지 않을 수 있습니다.")
else:
    print("두 그룹의 분산이 통계적으로 유의미하게 다르지 않습니다. 등분산 가정을 만족할 수 있습니다.")


Levene's test statistic: 0.07356153387697766
p-value: 0.7880180080011621
두 그룹의 분산이 통계적으로 유의미하게 다르지 않습니다. 등분산 가정을 만족할 수 있습니다.


### t-test

In [13]:
# 실제로 이 평균값이 유의미한가를 t-test를 통해 확인


# (귀무가설) : 오프라인이면서 manufacturer인 사람중에서 제품 이름길이에 따른 매출평균 차이가 없을것.
# (대립가설) : 오프라인이면서 manufacturer인 사람중에서 제품 이름길이에 따른 매출평균 차이가 있을것.

group1_sales = list(a[a.product_name_length>52].groupby('product_category_name')['price_total'].mean())  # 사진 수가 1인 그룹의 매출 데이터
group2_sales = list(a[a.product_name_length<=52].groupby('product_category_name')['price_total'].mean())  

# 등분산 가정을 확인하고 독립표본 T-test 수행
statistic, p_value = scipy.stats.ttest_ind(group1_sales, group2_sales, equal_var=True)  # equal_var=False: 등분산 가정이 만족되지 않을 경우 Welch's t-test 사용

# 결과 출력
print("T-statistic:", statistic)
print("p-value:", p_value)

if p_value < 0.05:
    print("귀무 가설을 기각하고 대립 가설을 채택합니다.")
else:
    print("귀무 가설을 채택합니다.")

T-statistic: -0.06454999767761055
p-value: 0.9489467609159992
귀무 가설을 채택합니다.


In [14]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=group1_sales, name='제품이름길이<=52', histnorm='density', marker_color='#4A55A2'))
fig.add_trace(go.Histogram(x=group2_sales, name='제품이름길이>52', histnorm='density', marker_color='#A0BFE0'))

fig.update_layout(
    xaxis_title='매출',
    yaxis_title='확률 밀도',
    title='두 그룹의 매출 분포',
    width=800,
    barmode='stack',
)

fig.show()

In [15]:
fig = go.Figure()

# Scatter Trace 추가
fig.add_trace(go.Box(y=group1_sales,name='제품설명길이<=566', marker_color='#4A55A2'))
fig.add_trace(go.Box(y=group2_sales,name='제품설명길이>566', marker_color='#A0BFE0'))
fig.update_layout(
    xaxis_title='제품설명길이',
    yaxis_title='매출',
    title='두 그룹의 매출 분포',
    width=800,
)
fig.show()